In [2]:
import pandas as pd

In [3]:
train_labels = pd.read_csv("train_labels.csv")
train_values = pd.read_csv("train_values.csv")

In [6]:
train_values.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [4]:
from sklearn import preprocessing

In [5]:
encoded_df = train_values[['legal_ownership_status', 'land_surface_condition', 'foundation_type', 'roof_type',
              'ground_floor_type', 'other_floor_type', 'position',
              'plan_configuration']]

In [6]:
# le.fit(encoded_df)
encoded_df = encoded_df.apply(preprocessing.LabelEncoder().fit_transform)

In [7]:
train_values[['legal_ownership_status', 'land_surface_condition', 'foundation_type', 'roof_type',
              'ground_floor_type', 'other_floor_type', 'position',
              'plan_configuration']] = encoded_df

In [ ]:
# X = train_values
# y = train_labels["damage_grade"]

In [8]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.model_selection import StratifiedKFold , cross_val_score, cross_val_predict
from sklearn.metrics import fbeta_score, make_scorer

In [10]:
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
log = LogisticRegression()
ada = AdaBoostClassifier(base_estimator=rf)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels["damage_grade"]) 
# ada.fit(X_train,y_train).predict(X_test)

In [12]:
# cross_val_score(ada , X=train_values, y=train_labels["damage_grade"], cv=10, scoring="f1_micro")

In [13]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7)

In [14]:
# cross_val_score(knn , X=train_values, y=train_labels["damage_grade"], cv=10, scoring="f1_micro")

In [15]:
from tpot import TPOTClassifier

In [16]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5, verbosity=2,scoring="f1_micro")

In [17]:
pipeline_optimizer.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.7161166812140123
Generation 2 - Current best internal CV score: 0.7161166812140123
Generation 3 - Current best internal CV score: 0.7187157980770419
Generation 4 - Current best internal CV score: 0.7187157980770419
Generation 5 - Current best internal CV score: 0.7187157980770419

Best pipeline: DecisionTreeClassifier(MaxAbsScaler(ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.5, min_samples_leaf=8, min_samples_split=16, n_estimators=100)), criterion=gini, max_depth=5, min_samples_leaf=11, min_samples_split=6)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=20,
               random_state=None, scoring='f1_micro', subsample=1.0,
               template=None, use_dask=False, verbosity=2, warm_start=False)

In [26]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.ensemble import ExtraTreesClassifier


In [25]:
print(pipeline_optimizer.score(X_test, y_test))

0.7230587404644595


In [27]:
pipeline_optimizer.export('exported.py')